# Constructing dataset of fire incidences and their environmental predictors

Author: Jatan Buch (v1, 231120) 


In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
from numpy.lib.stride_tricks import sliding_window_view
from scipy import stats, interpolate
from scipy.optimize import minimize
from scipy.special import gamma
from sklearn.linear_model import LinearRegression
from math import factorial
import itertools

import netCDF4 # module that reads in .nc files (built on top of HDF5 format)
import pandas as pd
import geopandas as gpd
from geopandas.tools import sjoin
import xarray
import rioxarray
import xesmf as xe # for regridding
import pickle # for saving and loading models
from pathlib import Path

# CDS API
import cdsapi

# To map GRIB files to NetCDF Common Data Model
import cfgrib

from shapely.geometry import Point, mapping, LineString
from shapely.geometry.polygon import Polygon
from pyproj import CRS, Transformer # for transforming projected coordinates to elliptical coordinates
import cartopy.crs as ccrs # for defining and transforming coordinate systems
import cartopy.feature as cfeature # to add features to a cartopy map
import cartopy.io.shapereader as shpreader
from shapely.geometry import Point, mapping
from shapely.geometry.polygon import Polygon
from pyproj import CRS, Transformer # for transforming projected coordinates to elliptical coordinates

from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from calendar import monthrange
from cftime import num2date, date2num, DatetimeGregorian
from tqdm import tqdm

import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib.patches import Rectangle
import matplotlib.patches as patches
import matplotlib.path as mpltPath
from plot_params_mpl39 import params
import matplotlib.pylab as pylab
import seaborn as sns
pylab.rcParams.update(params)
cols_default = plt.rcParams['axes.prop_cycle'].by_key()['color']

%matplotlib inline
%config IPython.matplotlib.backend = 'retina'
%config InlineBackend.figure_format = 'retina'

## Vegetation data

In [2]:
casm_ds= xarray.open_dataset('../data/casm_for_jatan.nc', chunks={"date": 10})

In [7]:
casm_ds.CASM_soil_moisture

<xarray.DataArray 'CASM_soil_moisture' (date: 2210, lat: 125, lon: 350)>
dask.array<open_dataset-CASM_soil_moisture, shape=(2210, 125, 350), dtype=float64, chunksize=(10, 125, 350), chunktype=numpy.ndarray>
Coordinates:
  * date     (date) datetime64[ns] 2002-06-19 2002-06-22 ... 2020-08-31
  * lat      (lat) float64 30.07 30.29 30.52 30.75 ... 68.33 68.87 69.41 69.97
  * lon      (lon) float64 -155.0 -154.7 -154.5 -154.2 ... -60.56 -60.3 -60.04

In [16]:
csif_ds= xarray.open_dataset('../data/csif_for_jatan.nc', chunks={"date": 10})

In [18]:
csif_ds.clear_daily_SIF

<xarray.DataArray 'clear_daily_SIF' (time: 1699, lat: 800, lon: 1900)>
dask.array<open_dataset-clear_daily_SIF, shape=(1699, 800, 1900), dtype=float32, chunksize=(1699, 800, 1900), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float64 30.02 30.07 30.12 30.17 ... 69.82 69.87 69.92 69.97
  * lon      (lon) float64 -155.0 -154.9 -154.9 -154.8 ... -60.12 -60.07 -60.02
  * time     (time) datetime64[ns] 2002-01-01 2002-01-05 ... 2020-08-08

## ERA5 reanalysis data

(Used instead of gridMET for future compatability with Canadian/Australian data as well)

In [2]:
URL = 'https://cds.climate.copernicus.eu/api/v2'
KEY = '136899:e611201e-7672-4596-9783-4fc89f7161e3'

DATADIR = '../data/meteorology'
sub = (51.6, -128, 26.5, -101) # North/West/South/East
ds_out = xarray.Dataset(
    {
        "lat": (["lat"], np.arange(26.5, 51.6, 0.125), {"units": "degrees_north"}),
        "lon": (["lon"], np.arange(-128, -101, 0.125), {"units": "degrees_west"}),
    }
)

In [3]:
var_arr= ['2m_dewpoint_temperature', '2m_temperature', 'total_precipitation'] #'10m_u_component_of_wind', '10m_v_component_of_wind',
yr_arr= list(np.arange(2002, 2009).astype(str))

In [5]:
c = cdsapi.Client(url=URL, key=KEY)
for var in var_arr:
    for yr in yr_arr:
        c.retrieve(
            'reanalysis-era5-single-levels',
            {
                'product_type': 'reanalysis',
                'format': 'grib',
                'variable': [var],
                'year': [yr],
                'month': ['02', '03', '04', '05', '06', '07', '08', '09'],
                'day': [
                    '01', '02', '03',
                    '04', '05', '06',
                    '07', '08', '09',
                    '10', '11', '12',
                    '13', '14', '15',
                    '16', '17', '18',
                    '19', '20', '21',
                    '22', '23', '24',
                    '25', '26', '27',
                    '28', '29', '30',
                    '31',
                ],
                'time': [
                    '00:00', '01:00', '02:00',
                    '03:00', '04:00', '05:00',
                    '06:00', '07:00', '08:00',
                    '09:00', '10:00', '11:00',
                    '12:00', '13:00', '14:00',
                    '15:00', '16:00', '17:00',
                    '18:00', '19:00', '20:00',
                    '21:00', '22:00', '23:00',
                ],
                'area': [51.6, -128, 26.5, -101],
            },
            f'{DATADIR}/era5_wus_forecast_hourly_%s'%var + f'_%s.grib'%yr)

2023-11-20 15:08:21,084 INFO Welcome to the CDS
2023-11-20 15:08:21,084 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2023-11-20 15:08:21,292 INFO Request is queued
2023-11-20 15:46:48,827 INFO Request is running
2023-11-20 15:50:49,767 INFO Request is completed
2023-11-20 15:50:49,769 INFO Downloading https://download-0000-clone.copernicus-climate.eu/cache-compute-0000/cache/data2/adaptor.mars.internal-1700513134.7476625-32345-8-5dc7377e-a3d6-4313-9c7d-8ebf69db0628.grib to ../data/meteorology//era5_wus_forecast_hourly_2m_dewpoint_temperature_2010.grib (123M)
2023-11-20 15:51:04,585 INFO Download rate 8.3M/s  
2023-11-20 15:51:04,817 INFO Welcome to the CDS
2023-11-20 15:51:04,818 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2023-11-20 15:51:04,959 INFO Request is queued
2023-11-20 15:51:07,715 INFO Request is running
2023-11-20 15:57:24,773 INFO Request is completed
2

In [27]:
ds= xarray.open_dataset(f'{DATADIR}/era5_wus_forecast_hourly_2m_temperature.grib', engine='cfgrib')